In [25]:
import pandas as pd
import numpy as np
import re

In [4]:
df = pd.read_csv('data/df_cleaned_20250617.csv')

print("Preprocessing dataset...")
df['latitude'] = df['center_latlon'].apply(lambda x: str(x.split(', ')[0]))
df['longitude'] = df['center_latlon'].apply(lambda x: str(x.split(', ')[1]))
df['latitude'] = df['latitude'].apply(lambda x: float(re.search(r'\d+.\d+', x).group()))
df['longitude'] = df['longitude'].apply(lambda x: float(re.search(r'\-\d+.\d+', x).group()))
df['timestamp_sec'] = pd.to_datetime(df['timestamp'])
df['timestamp_sec'] = (df['timestamp_sec'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

# Separate training data
df_train = df.dropna(subset=['ground_truth'])
df_test = df[df['ground_truth'].isna()]

Preprocessing dataset...


In [46]:
df.head()

,timestamp,bboxid,GEOID,geometry_bbox,center_latlon,max,min,precipitation,total_population,white_ratio,black_ratio,hh_median_income,ground_truth,latitude,longitude,timestamp_sec
0,2016-01-01,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",50.0,40.0,0.00,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353,1451606400
1,2016-01-02,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",51.0,45.0,0.00,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353,1451692800
2,2016-01-03,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",55.0,46.0,0.02,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353,1451779200
3,2016-01-04,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",57.0,47.0,0.09,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353,1451865600
4,2016-01-05,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",54.0,50.0,1.18,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353,1451952000


In [51]:
print(len(df['timestamp'].unique()))
print(len(df['bboxid'].unique()))

3074
4379


In [54]:
print(df['timestamp'].max())
print(df['timestamp'].min())

2024-05-31
2016-01-01


In [56]:
3074*4379

13461046

In [55]:
df.shape

(13461046, 16)

In [49]:
df[df['ground_truth']>=1].shape

(228183, 16)

In [47]:
df['ground_truth'].value_counts()

ground_truth
1.0     178818
0.0      43809
2.0      33816
3.0       9903
4.0       3442
5.0       1252
6.0        524
7.0        233
8.0         96
9.0         50
10.0        26
11.0         9
12.0         7
13.0         5
15.0         2
Name: count, dtype: int64

In [30]:
len(df['timestamp_sec'].unique())

3074

In [34]:
df_train.groupby('bboxid')['ground_truth'].mean().reset_index()

,bboxid,ground_truth
0,bbox_10_14,0.333333
1,bbox_10_15,0.000000
2,bbox_10_16,0.000000
3,bbox_10_17,0.500000
4,bbox_10_18,0.000000
...,...,...
4374,bbox_9_64,0.666667
4375,bbox_9_65,0.666667
4376,bbox_9_66,0.666667
4377,bbox_9_67,0.500000


In [41]:
bbox_counts = df_train.groupby('bboxid')['ground_truth'].mean().reset_index()
bboxids = df_train['bboxid'].values

# Top k bounding boxes by density (tent count)
top10 = bbox_counts.nlargest(10, 'ground_truth')['bboxid'].values
top10


array(['bbox_26_1', 'bbox_41_43', 'bbox_53_47', 'bbox_57_27', 'bbox_5_47',
       'bbox_45_60', 'bbox_57_50', 'bbox_44_60', 'bbox_43_45',
       'bbox_57_49'], dtype=object)

In [40]:
np.where(df_train['bboxid'].isin(top10))[0]

array([ 59769,  59770,  59771, ..., 262053, 262054, 262055])

In [45]:
8%3

2

In [42]:
np.where(np.isin(bboxids, top10))[0]

array([ 59769,  59770,  59771, ..., 262053, 262054, 262055])

In [5]:
df_train.head()

,timestamp,bboxid,GEOID,geometry_bbox,center_latlon,max,min,precipitation,total_population,white_ratio,black_ratio,hh_median_income,ground_truth,latitude,longitude,timestamp_sec
313,2016-11-09,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",73.0,58.0,0.0,506,0.806324,0.000000,53438.0,0.0,37.719862,-122.485353,1478649600
314,2016-11-10,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",74.0,60.0,0.0,506,0.806324,0.000000,53438.0,0.0,37.719862,-122.485353,1478736000
2141,2016-11-10,bbox_10_15,60750332043,POLYGON ((-122.48444469610592 37.7191324443233...,"(37.71985369613769, -122.4835266585265)",74.0,60.0,0.0,1852,0.266199,0.075054,77813.0,0.0,37.719854,-122.483527,1478736000
3967,2016-11-09,bbox_10_16,60750332043,POLYGON ((-122.48261869150517 37.7191244450116...,"(37.719845682477825, -122.48170063648698)",73.0,58.0,0.0,1852,0.266199,0.075054,77813.0,0.0,37.719846,-122.481701,1478649600
3968,2016-11-10,bbox_10_16,60750332043,POLYGON ((-122.48261869150517 37.7191244450116...,"(37.719845682477825, -122.48170063648698)",74.0,60.0,0.0,1852,0.266199,0.075054,77813.0,0.0,37.719846,-122.481701,1478736000


In [22]:
9 // 5

1

In [ ]:
df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])

# Extracting year, month, day
df_train['year'] = df_train['timestamp'].dt.year
df_train['month'] = df_train['timestamp'].dt.month
df_train['day'] = df_train['timestamp'].dt.day

/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_55136/4101039952.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['timestamp'] = pd.to_datetime(df_train['timestamp'])
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_55136/4101039952.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['year'] = df_train['timestamp'].dt.year
/var/folders/y6/z9h695vx2w3gbjh7z32h4j9m0000gn/T/ipykernel_55136/4101039952.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [16]:
# Group by year and count occurrences

df_yearly_counts = df_train.groupby(['year']).size().reset_index(name='count')
df_yearly_counts

,year,count
0,2016,32410
1,2017,37597
2,2018,56404
3,2019,43336
4,2020,31729
5,2021,19420
6,2022,20779
7,2023,22372
8,2024,7945


In [ ]:
def make_time_forward_splits(year, n_splits=3, horizon_years=1):
    """
    Create time-forward splits for cross-validation.
    """
    unique_years = np.sort(np.unique(year)) #2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
    splits = []
    max_idx = len(unique_years) - horizon_years
    block = max_idx // n_splits

    for k in range(n_splits):
        train_end = unique_years[(k+1)*block-1] #block=2, train_end=2017, 2019, 2021
        val_start = unique_years[(k+1)*block] #val_start=2018, 2020, 2022
        val_end = val_start + horizon_years #val_end=2019, 2021, 2023
        train_idx = np.where(year <= train_end)[0]
        val_idx = np.where((year > train_end) & (year <= val_end))[0]
        splits.append((train_idx, val_idx))
    
    return splits

In [29]:
make_time_forward_splits(df_train['year'].values, n_splits=3, horizon_years=2)

[(array([     0,      1,      2, ..., 201473, 201474, 201475]),
  array([    15,     16,     17, ..., 201469, 201470, 201472])),
 (array([     0,      1,      2, ..., 201473, 201474, 201475]),
  array([    23,     40,     65, ..., 271987, 271990, 271991])),
 (array([     0,      1,      2, ..., 271955, 271956, 271987]),
  array([201476, 201482, 201484, ..., 271989, 271990, 271991]))]

In [18]:
df_train['ground_truth'].value_counts()

ground_truth
1.0     178818
0.0      43809
2.0      33816
3.0       9903
4.0       3442
5.0       1252
6.0        524
7.0        233
8.0         96
9.0         50
10.0        26
11.0         9
12.0         7
13.0         5
15.0         2
Name: count, dtype: int64

In [10]:
df_test.head()

,timestamp,bboxid,GEOID,geometry_bbox,center_latlon,max,min,precipitation,total_population,white_ratio,black_ratio,hh_median_income,ground_truth,latitude,longitude
0,1451606400,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",50.0,40.0,0.00,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353
1,1451692800,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",51.0,45.0,0.00,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353
2,1451779200,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",55.0,46.0,0.02,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353
3,1451865600,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",57.0,47.0,0.09,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353
4,1451952000,bbox_10_14,60750604002,POLYGON ((-122.4862707014277 37.71914041535512...,"(37.71986168151697, -122.48535268128838)",54.0,50.0,1.18,506,0.806324,0.0,53438.0,NaN,37.719862,-122.485353


In [13]:
# Check for missing values in df_test except 'ground_truth'
missing_values = df_test.drop(columns=['ground_truth']).isnull().sum()
print("Missing values in df_test:", missing_values)

Missing values in df_test: timestamp           0
bboxid              0
GEOID               0
geometry_bbox       0
center_latlon       0
max                 0
min                 0
precipitation       0
total_population    0
white_ratio         0
black_ratio         0
hh_median_income    0
latitude            0
longitude           0
dtype: int64
